In [21]:
import os
import sys

import numpy as np

from torch.utils.data import DataLoader
from tqdm import tqdm

sys.path.append('../src/')

from dataset.ai_synth_dataset import AiSynthDataset
from model.lit_module import LitModularSynth
from utils.train_utils import get_project_root, to_numpy_recursive

# Evaluate a sound matching model

In this notebook we are going over the steps to evaluate a sound matching model on a given test dataset.
To see more about training such a model, see *train_model.ipynb*

<br>

Assumptions:
1. The test data is generated by DiffMoog using the same preset as the one used for training the model (see *create_dataset.ipynb*)
2. The resulting metrics will be the same as the ones reported during training. To add metrics edit *LitModularSynth._calculate_audio_metrics()*

In [2]:
root = get_project_root()  # Verify correct project root or insert manually

exp_name = "basic_flow_example_experiment"
device = 'cuda:0'

ckpt_name = r'epoch=1-step=80.ckpt'
ckpt_path = root.joinpath('experiments', 'current', exp_name, 'checkpoints', ckpt_name)   # Path to the requested checkpoint

synth_module = LitModularSynth.load_from_checkpoint(ckpt_path, device=device).to(device)

In [3]:
val_data_dir = os.path.join(synth_module.cfg.data_dir, 'val')  # You can input a data directory manually instead

val_dataset = AiSynthDataset(val_data_dir)
dataloader = DataLoader(val_dataset, batch_size=64)

In [24]:
from IPython.utils.io import capture_output
import torch

all_losses, all_metrics = [], []

for batch in tqdm(dataloader):

    batch[0] = batch[0].to(device)

    with torch.no_grad(), capture_output():
        loss, step_losses, step_metrics, step_artifacts = synth_module.in_domain_step(batch, return_metrics=True)

    all_losses.append(step_losses)
    all_metrics.append(step_metrics)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.52it/s]


In [25]:
all_losses = {k: np.mean([to_numpy_recursive(dic[k]) for dic in all_losses]) for k in all_losses[0]}
all_metrics = {k: np.mean([to_numpy_recursive(dic[k]) for dic in all_metrics]) for k in all_metrics[0]}

for title, metrics_dict in zip(['Losses', 'Metrics'], [all_losses, all_metrics]):
    print(f'\n{title}:')
    for metric_name, metric_val in metrics_dict.items():
        print(f"\t{metric_name}: {metric_val}")
    print("************************************************")


Losses
	raw_params_loss: 7.486570358276367
	raw_spec_loss: 0.0
	weighted_params_loss: 7.486570358276367
	weighted_spec_loss: 0.0
	loss_total: 7.486570566892624
************************************************

Metrics
	paper_lsd_value: 29.058761596679688
	lsd_value: 15.86365795135498
	pearson_stft: 0.08646538108587265
	pearson_fft: 0.05919661745429039
	mean_average_error: 1.5392720699310303
	mfcc_mae: 19.468284606933594
	spectral_convergence_value: 20.77668571472168
************************************************
